### Counterfactual Regret Minimization (CFR) and its application to Kuhn Poker


Source consulted: https://modelai.gettysburg.edu/2013/cfr/cfr.pdf


Kuhn Poker is a simple 3-card poker game created by Harold E. Kuhn. Two players each bet 1 chip blind into the pot before the deal. Three cards (usually K, Q, and J) are suffled, and one card is dealt to each player and held as private information in the original Kuhn Poker game. We implement the game as described in the above paper with additional change as inspired by the [ReBeL](https://arxiv.org/abs/2007.13544) paper. Players do not have any private information -- a referee observes the players' cards and makes decision based on the players' strategy which they announce in the beginning of the game. The players then update their belief based on how the other player's play and simultaneously update their strategy. Further information can be found on the paper.  

In [24]:
import numpy as np
import random
from typing import List, Dict

In [25]:
class Kuhn_Poker():
    """
    Actions: Pass (0)
             Bet  (1)
             
    Number of Actions: 2
    
    """
    @staticmethod
    def is_terminal(history):
        return history in ['00', '11', '10', '01']
        
    @staticmethod
    def get_payoff(history, cards):
        
        """
        payoff structure:
        
            1) check if both players have had at least one action:
                 i) a 'terminal' pass after the first action:
                     a) if it is a terminal pass, then a double terminal pass gives 1 chip to 
                         the player with higher card
                     b) if it is single pass after a bet, the player betting wins 1 chip
                 
                 ii) if not terminal pass, but two consecutive bets, then player with higher card
                     gets 2 chips
    
        """
        
        plays = len(history)
        player = len(history[plays-1]) % 2
        op = 1 - player
        
        print(history, plays)
        terminal_pass = history[plays-1] == '0'
        double_bet = history == '11'
        if terminal_pass:
            if history == '00':
                return 1  if int(cards[player]) > int(cards[op])  else -1
            else:
                return 1

        elif double_bet:
            return 2 if int(cards[player]) > int(cards[op]) else -2    

In [26]:
class Info_Set():
    """
    Action-observation history of the game
    
    """
    
    def __init__(self):
        self.regret_sum = np.zeros(shape=2)
        self.strategy = np.zeros(shape=2)
        self.strategy_sum = np.zeros(shape=2) 
    
    def normalize(self):
        """
        Normalize the strategy to (0, 1). If the regrets are negative, return uniform strategy.
        
        """
        if sum(self.strategy) > 0:
            self.strategy /= np.sum(self.strategy)
        else:
            self.strategy = np.ones(2)/2
        
        return self.strategy
    
    def get_strategy(self, reach_prob):
        self.strategy = np.maximum(0, self.regret_sum)
        self.normalize() 
        
        self.strategy_sum = reach_prob * self.strategy
        
        return self.strategy
    
    def get_average_strategy(self):
        return self.strategy_sum/np.sum(self.strategy_sum)

In [33]:
class CFRTrainer():
    def __init__(self):
        self.infoset_map: Dict[str, Info_Set] = {}
            
    def get_info_set(self, history):
        """
        return the history while adding if needed
        
        """
        if history not in self.infoset_map:
            self.infoset_map[history] = Info_Set()
            
        return self.infoset_map[history]
    
    def cfr(self, cards, history, reach_prob, current_player):
        if Kuhn_Poker.is_terminal(history):
            return Kuhn_Poker.get_payoff(history, cards)
        
        my_card = cards[current_player]
        info_set = self.get_info_set(history + my_card)
        
        strategy = info_set.get_strategy(reach_prob[current_player])
        op = (current_player + 1)%2
        cfr_values = np.zeros(2)
        
        for action in range(2):
            action_prob = strategy[action]
            
            # compute new_reach_probabilities for this action
            new_reach_prob = reach_prob.copy()
            new_reach_prob[current_player] *= action_prob
            
            # call CFR recursively with next player to act as opponent
            
            cfr_values[action] = -1*self.cfr(cards, history + str(action), new_reach_prob, op)
            
        # value of the current node is counterfactual values * action probabilities
        node_value = cfr_values.dot(strategy)
        
        for action in range(2):
            info_set.regret_sum += reach_prob[op] * (cfr_values[action] - node_value)
        
        return node_value     
    
    def train(self, num_iter):
        util = 0
        K_cards = ['0', '1', '2'] # J, Q, K
        for _ in range(num_iter):
            cards = random.sample(K_cards, 2)
            history = ''
            reach_prob = np.ones(2)
            util += self.cfr(cards, history, reach_prob, 0)
        return util

In [38]:
from typing import List, Dict
import random
import numpy as np
import sys

Actions = ['B', 'C']  # bet/call vs check/fold

class InformationSet():
    def __init__(self):
        self.cumulative_regrets = np.zeros(shape=len(Actions))
        self.strategy_sum = np.zeros(shape=len(Actions))
        self.num_actions = len(Actions)

    def normalize(self, strategy: np.array) -> np.array:
        """Normalize a strategy. If there are no positive regrets,
        use a uniform random strategy"""
        if sum(strategy) > 0:
            strategy /= sum(strategy)
        else:
            strategy = np.array([1.0 / self.num_actions] * self.num_actions)
        return strategy

    def get_strategy(self, reach_probability: float) -> np.array:
        """Return regret-matching strategy"""
        strategy = np.maximum(0, self.cumulative_regrets)
        strategy = self.normalize(strategy)

        self.strategy_sum += reach_probability * strategy
        return strategy

    def get_average_strategy(self) -> np.array:
        return self.normalize(self.strategy_sum.copy())


class KuhnPoker():
    @staticmethod
    def is_terminal(history: str) -> bool:
        return history in ['BC', 'BB', 'CC', 'CBB', 'CBC']

    @staticmethod
    def get_payoff(history: str, cards: List[str]) -> int:
        """get payoff for 'active' player in terminal history"""
        if history in ['BC', 'CBC']:
            return +1
        else:  # CC or BB or CBB
            payoff = 2 if 'B' in history else 1
            active_player = len(history) % 2
            player_card = cards[active_player]
            opponent_card = cards[(active_player + 1) % 2]
            if player_card == 'K' or opponent_card == 'J':
                return payoff
            else:
                return -payoff


class KuhnCFRTrainer():
    def __init__(self):
        self.infoset_map: Dict[str, InformationSet] = {}

    def get_information_set(self, card_and_history: str) -> InformationSet:
        """add if needed and return"""
        if card_and_history not in self.infoset_map:
            self.infoset_map[card_and_history] = InformationSet()
        return self.infoset_map[card_and_history]

    def cfr(self, cards: List[str], history: str, reach_probabilities: np.array, active_player: int):
        if KuhnPoker.is_terminal(history):
            return KuhnPoker.get_payoff(history, cards)

        my_card = cards[active_player]
        info_set = self.get_information_set(my_card + history)

        strategy = info_set.get_strategy(reach_probabilities[active_player])
        opponent = (active_player + 1) % 2
        counterfactual_values = np.zeros(len(Actions))

        for ix, action in enumerate(Actions):
            action_probability = strategy[ix]

            # compute new reach probabilities after this action
            new_reach_probabilities = reach_probabilities.copy()
            new_reach_probabilities[active_player] *= action_probability

            # recursively call cfr method, next player to act is the opponent
            counterfactual_values[ix] = -self.cfr(cards, history + action, new_reach_probabilities, opponent)

        # Value of the current game state is just counterfactual values weighted by action probabilities
        node_value = counterfactual_values.dot(strategy)
        for ix, action in enumerate(Actions):
            info_set.cumulative_regrets[ix] += reach_probabilities[opponent] * (counterfactual_values[ix] - node_value)

        return node_value

    def train(self, num_iterations: int) -> int:
        util = 0
        kuhn_cards = ['J', 'Q', 'K']
        for _ in range(num_iterations):
            cards = random.sample(kuhn_cards, 2)
            history = ''
            reach_probabilities = np.ones(2)
            util += self.cfr(cards, history, reach_probabilities, 0)
        return util

In [39]:
num_iterations = 1000
cfr_trainer = KuhnCFRTrainer()
util = cfr_trainer.train(num_iterations)

print(f"\nRunning Kuhn Poker chance sampling CFR for {num_iterations} iterations")
print(f"\nExpected average game value (for player 1): {(-1./18):.3f}")
print(f"Computed average game value               : {(util / num_iterations):.3f}\n")

print("We expect the bet frequency for a Jack to be between 0 and 1/3")
print("The bet frequency of a King should be three times the one for a Jack\n")

print(f"History  Bet  Pass")
for name, info_set in sorted(cfr_trainer.infoset_map.items(), key=lambda s: len(s[0])):
    print(f"{name:3}:    {info_set.get_average_strategy()}")


Running Kuhn Poker chance sampling CFR for 1000 iterations

Expected average game value (for player 1): -0.056
Computed average game value               : -0.064

We expect the bet frequency for a Jack to be between 0 and 1/3
The bet frequency of a King should be three times the one for a Jack

History  Bet  Pass
K  :    [0.78358663 0.21641337]
J  :    [0.25143939 0.74856061]
Q  :    [0.06002032 0.93997968]
QB :    [0.42524923 0.57475077]
QC :    [0.01391304 0.98608696]
JB :    [0.00154799 0.99845201]
JC :    [0.33576608 0.66423392]
KB :    [0.99849398 0.00150602]
KC :    [0.99849398 0.00150602]
KCB:    [0.99646729 0.00353271]
JCB:    [9.48790609e-04 9.99051209e-01]
QCB:    [0.59656809 0.40343191]
